In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3045/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [7]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
19,model_22_9_7,0.999774,0.853455,0.999330,0.999280,0.999513,0.000321,0.208000,0.000595,0.000711,0.000653,0.054618,0.017928,1.000014,0.018692,842.085504,1345.481220,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
20,model_22_9_8,0.999773,0.853679,0.999223,0.999239,0.999462,0.000322,0.207681,0.000690,0.000751,0.000721,0.050860,0.017935,1.000014,0.018699,842.083945,1345.479661,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
21,model_22_9_9,0.999771,0.853879,0.999119,0.999201,0.999414,0.000325,0.207398,0.000783,0.000789,0.000786,0.047475,0.018033,1.000014,0.018801,842.062174,1345.457890,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
22,model_22_9_6,0.999771,0.853202,0.999440,0.999321,0.999564,0.000326,0.208358,0.000498,0.000670,0.000584,0.058791,0.018046,1.000014,0.018814,842.059295,1345.455011,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
23,model_22_9_10,0.999767,0.854056,0.999020,0.999164,0.999367,0.000331,0.207146,0.000871,0.000825,0.000848,0.044435,0.018195,1.000014,0.018969,842.026516,1345.422231,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
24,model_22_9_5,0.999763,0.852918,0.999550,0.999364,0.999616,0.000336,0.208762,0.000400,0.000628,0.000514,0.063435,0.018329,1.000015,0.019109,841.997168,1345.392883,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
28,model_22_9_4,0.999750,0.852596,0.999658,0.999406,0.999668,0.000354,0.209218,0.000304,0.000586,0.000445,0.068597,0.018821,1.000015,0.019622,841.891190,1345.286905,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
33,model_22_9_3,0.999730,0.852233,0.999759,0.999447,0.999717,0.000383,0.209733,0.000214,0.000546,0.000380,0.074851,0.019570,1.000017,0.020403,841.735029,1345.130745,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
41,model_22_9_2,0.999700,0.851823,0.999851,0.999485,0.999761,0.000425,0.210316,0.000132,0.000508,0.000320,0.082810,0.020624,1.000018,0.021502,841.525141,1344.920857,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
46,model_22_9_1,0.999658,0.851357,0.999926,0.999519,0.999798,0.000485,0.210978,0.000065,0.000475,0.000270,0.091642,0.022029,1.000021,0.022966,841.261665,1344.657381,"Hidden Size=[20, 16], regularizer=0.2, learnin..."
